In [3]:

from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")

data = loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
    add_start_index = True,)
texts = text_splitter.split_documents(data)



In [13]:
texts[0].dict()

{'page_content': "LLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nJune 23, 2023\xa0·\xa031 min\xa0·\xa0Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n\nComponent Three: Tool Use\n\nCase Studies\n\nScientific Discovery Agent\n\nGenerative Agents Simulation\n\nProof-of-Concept Examples\n\n\nChallenges\n\nCitation\n\nReferences",
 'metadata': {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
  'title': "LLM Powered Autonomous Agents | Lil'Log",
  'description': 'Building agents with LLM (large language model) as its core co

In [8]:
import pinecone

index_name = 'langchain-retrieval-augmentation-fast'
PINECONE_API_KEY = "6a8a1f2f-0066-4e9d-9695-8d4a44960617"
# find ENV (cloud region) next to API key in console
PINECONE_ENVIRONMENT = "gcp-starter"

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=1536,  # 1536 dim of text-embedding-ada-002
        
    )

In [9]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key="") # set openai_api_key = 'your_openai_api_key'


In [27]:
from supabase import create_client, Client

from langchain.vectorstores.supabase import SupabaseVectorStore


In [19]:
supabase: Client = create_client("https://crhertcomzmmxpsmqozi.supabase.co", "")

In [30]:
vector_store = await SupabaseVectorStore.afrom_documents(texts, embeddings, client=supabase, table_name="documents", query_name="match_documents", chunk_size=500)


2023-10-28 16:56:16,017:INFO - HTTP Request: POST https://crhertcomzmmxpsmqozi.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"


In [42]:
vector_store= SupabaseVectorStore(client=supabase,embedding=embeddings, table_name="documents",query_name="match_documents")
vector_store.similarity_search_with_relevance_scores("What did the president say about Ketanji Brown Jackson")


2023-10-28 17:25:49,884:INFO - HTTP Request: POST https://crhertcomzmmxpsmqozi.supabase.co/rest/v1/rpc/match_documents?limit=4 "HTTP/1.1 200 OK"


[(Document(page_content='System message:', metadata={'text': 'System message:', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'language': 'en', 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'start_index': 33554}),
  0.723420083522802),
 (Document(page_content='Conversatin samples:\n[\n  {\n    "role": "system",', metadata={'text': 'Conversatin samples:\n[\n  {\n    "role": "system",', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'source': 'https://lilia

In [46]:
vector_store= SupabaseVectorStore(client=supabase,embedding=embeddings, table_name="documents",query_name="match_documents_mmr")
vector_store.max_marginal_relevance_search("What did the president say about Ketanji Brown Jackson", k=4,fetch_k=20)


2023-10-28 17:31:58,314:INFO - HTTP Request: POST https://crhertcomzmmxpsmqozi.supabase.co/rest/v1/rpc/match_documents_mmr?limit=20 "HTTP/1.1 200 OK"
2023-10-28 17:31:58,736:INFO - Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.
2023-10-28 17:31:58,738:INFO - Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.
2023-10-28 17:31:58,739:INFO - Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.
2023-10-28 17:31:58,740:INFO - Unable to import simsimd, defaulting to NumPy implementation. If you want to use simsimd please install with `pip install simsimd`.


[Document(page_content='System message:', metadata={'text': 'System message:', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'language': 'en', 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'start_index': 33554}),
 Document(page_content='inquired about current trends in anticancer drug discovery;\nselected a target;\nrequested a scaffold targeting these compounds;\nOnce the compound was identified, the model attempted its synthesis.', metadata={'text': 'inquire

In [47]:
from langchain.memory import VectorStoreRetrieverMemory
retriever=vector_store.as_retriever(search_kwargs=dict(k=5))
memory = VectorStoreRetrieverMemory(retriever=retriever)




In [48]:
memory.save_context({"input": "My favorite food is pizza"}, {"output": "that's good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't the Celtics"}, {"output": "ok"}) #

2023-10-28 17:47:55,025:INFO - HTTP Request: POST https://crhertcomzmmxpsmqozi.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-10-28 17:47:56,474:INFO - HTTP Request: POST https://crhertcomzmmxpsmqozi.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"
2023-10-28 17:47:57,409:INFO - HTTP Request: POST https://crhertcomzmmxpsmqozi.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"


In [14]:
docsearch=Pinecone.from_existing_index(index_name, embeddings)

In [15]:
docsearch.max_marginal_relevance_search("What is LLM powered agents system",k=2,fetch_k=10)

[Document(page_content="LLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nJune 23, 2023\xa0·\xa031 min\xa0·\xa0Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n\nComponent Three: Tool Use\n\nCase Studies\n\nScientific Discovery Agent\n\nGenerative Agents Simulation\n\nProof-of-Concept Examples\n\n\nChallenges\n\nCitation\n\nReferences", metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiri

In [ ]:
docsearch.as_retriever()